In [8]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio

import os, h5py, sys
from pathlib import Path
import openpyxl

import holoviews as hv

In [9]:
%run -n C:\Users\christianl\repos\lwi-apps\src\utility_code.py

In [11]:
model=ras.get_model_paths('../../simulation')

 ### get_manning_data

In [ ]:
# set location of hdf data
paln_hdf_loc = r'../../simulation/WF_WestForkCalcasieu.p14.hdf'
geo_hdf_loc = r'../../simulation/WF_WestForkCalcasieu.g03.hdf'
flow_hdf_loc = r'../../simulation/WF_WestForkCalcasieu.u10.hdf'

In [ ]:
ras.get_manning_data(geo_hdf_loc).head()

 ### get_model_paths
 
 This function needs to take the typical RAS folder structure and return the relevant plan associations for each hec ras plan

In [ ]:
# %cd ../../simulation

In [ ]:
# %pwd

In [ ]:
# path = %pwd
# path=Path(path)

In [ ]:
model_path = '../../simulation/'
model_path=Path(model_path)

In [ ]:
for child in Path(model_path).glob('*.prj'):
    if child.is_file():
        prj_file=child

prj_file

In [ ]:
prj_file_list=(Path.read_text(prj_file).split("\n"))
plan_list = [f'.{x.split("=")[-1]}' for x in prj_file_list if x.startswith("Plan File=")]
plan_list[:5]

In [ ]:
# find all files whose extension is in plan_list return the full path
plan_file_list = [x for x in Path(model_path).glob('*') if x.suffix in plan_list and x.stem != "Backup"]
plan_file_list

In [ ]:
# create a dictionary of plan names and file paths the name is found in the plan file
plan_dict = {}
for plan_file in plan_file_list:
    plan_name = Path.read_text(plan_file).split("\n")[0].split("=")[-1]
    plan_dict[plan_name] = plan_file

# create a dataframe of plan names and file paths
plan_df = pd.DataFrame.from_dict(plan_dict, orient='index', columns=['plan_file'])
plan_df

In [ ]:
# Path.read_text(plan_dict['002YR']).split("\n")

In [ ]:
# add the plans geometry file to plan_df
plan_df['geom_file'] = plan_df['plan_file'].apply(lambda x: Path.read_text(x).split("\n")[4].split("=")[-1])
plan_df.head()

In [ ]:
for x in plan_df['geom_file']:
    print(x)

In [ ]:
for x in plan_df['geom_file']:
    for y in Path(model_path).glob(f'*{x}'):
        if y.stem != "Backup":
            print(y)

In [ ]:
prj_file.stem

In [ ]:
# use glob to find the geometry file and add it to plan_df
plan_df['geom_file'] = plan_df['geom_file'].apply(lambda x: [y for y in Path(model_path).glob(f'*{prj_file.stem}.{x}')][0] if y.stem != "Backup" else None)

In [ ]:
plan_df

In [ ]:
for child in Path(model_path).glob('*.hdf'):
    if child.is_file():
        print(child)

In [ ]:
# check if plan file has an hdf file associated with it and add it to the dataframe
for x in Path(model_path).glob('*.p*.hdf'):
    print(x.stem)

In [ ]:
plan_df

In [ ]:
# add hdf file extension to geom_file if it exists
plan_df['geom_file_hdf'] = plan_df['geom_file'].apply(lambda x: Path(f'{x}.hdf') if Path(f'{x}.hdf').exists() else None)
plan_df

In [ ]:
# add hdf file extension to plan_file if it exists
plan_df['plan_file_hdf'] = plan_df['plan_file'].apply(lambda x: Path(f'{x}.hdf') if Path(f'{x}.hdf').exists() else None)
plan_df

In [ ]:
cTRL01_Laura2020_B = plan_df.loc['Laura 2020']

In [ ]:
cTRL01_Laura2020_B['plan_file']

In [ ]:
# define the function to get the file dataframe from the model path
def get_model_info(model_path):
    # find the projection file
    for child in Path(model_path).glob('*.prj'):
        if child.is_file():
            prj_file=child

    # read the project file and get the plan file extension
    prj_file_list=(Path.read_text(prj_file).split("\n"))
    plan_list = [f'.{x.split("=")[-1]}' for x in prj_file_list if x.startswith("Plan File=")]

    # find all files whose extension is in plan_list return the full path
    plan_file_list = [x for x in Path(model_path).glob('*') if x.suffix in plan_list]

    # create a dictionary of plan names and file paths the name is found in the plan file
    plan_dict = {}
    for plan_file in plan_file_list:
        plan_name = Path.read_text(plan_file).split("\n")[0].split("=")[-1]
        plan_dict[plan_name] = plan_file

    # create a dataframe of plan names and file paths
    plan_df = pd.DataFrame.from_dict(plan_dict, orient='index', columns=['plan_file'])

    # add the plans geometry file to plan_df
    plan_df['geom_file'] = plan_df['plan_file'].apply(lambda x: Path.read_text(x).split("\n")[4].split("=")[-1])

    # use glob to find the geometry file
    plan_df['geom_file'] = plan_df['geom_file'].apply(lambda x: [y for y in Path(model_path).glob(f'*{x}')][0])

    # add hdf file extension to geom_file if it exists
    plan_df['geom_file_hdf'] = plan_df['geom_file'].apply(lambda x: Path(f'{x}.hdf') if Path(f'{x}.hdf').exists() else None)

    # add hdf file extension to plan_file if it exists
    plan_df['plan_file_hdf'] = plan_df['plan_file'].apply(lambda x: Path(f'{x}.hdf') if Path(f'{x}.hdf').exists() else None)

    return plan_df

In [ ]:
get_model_info('../../simulation/')

 ### get_sim_stats

 This function needs to return runtimes and volume error in acre feet and percentage for each plan in the model

 ### edit_manning_values

In [13]:
test_goemetry_path=model.geom_file.ctrl01_laura_2020_b
test_goemetry_path

WindowsPath('../../simulation/WF_WestForkCalcasieu.g04')

In [21]:
test_geo=Path.read_text(test_goemetry_path)
test_geo=list(test_geo.split("\n"))
# create a dataframe of the geometry file
test_geo_df=pd.DataFrame(test_geo)
# find rows between the start and end of the mannings n values and create a dataframe, they will start with "LCMann Region Name" and end with "LCMann Region Polygon"
test_geo_df[test_geo_df[0].str.contains("LCMann Region Name")].index[0]

154029

In [22]:
test_geo_df[test_geo_df[0].str.contains("LCMann Region Polygon")].index[0]

154048

In [38]:
# return each index that starts with "LCMann Region Name"
starts=test_geo_df[test_geo_df[0].str.contains("LCMann Region Name")].index
# return each index that ends with "LCMann Region Polygon"
ends=test_geo_df[test_geo_df[0].str.contains("LCMann Region Polygon")].index
print(f'Mannings desc (start) count: {len(test_geo_df[test_geo_df[0].str.contains("LCMann Region Name")].index)}\nMannings desc (end) count: {len(test_geo_df[test_geo_df[0].str.contains("LCMann Region Polygon")].index)}')

Mannings desc (start) count: 119
Mannings desc (end) count: 119


In [42]:
#  zip the start and end indexes together
mannings_index=list(zip(starts,ends))
mannings_index

[(154029, 154048),
 (154584, 154603),
 (155272, 155291),
 (155925, 155944),
 (156510, 156529),
 (156790, 156809),
 (157633, 157652),
 (158898, 158917),
 (159533, 159552),
 (159819, 159838),
 (160426, 160445),
 (160766, 160785),
 (161443, 161462),
 (161524, 161543),
 (161760, 161779),
 (162388, 162407),
 (162956, 162975),
 (163122, 163141),
 (163227, 163246),
 (163534, 163553),
 (163906, 163925),
 (164445, 164464),
 (164985, 165004),
 (165372, 165391),
 (167545, 167564),
 (167724, 167743),
 (167912, 167931),
 (168038, 168057),
 (168170, 168186),
 (168208, 168224),
 (169594, 169610),
 (170123, 170139),
 (170548, 170564),
 (171059, 171075),
 (171736, 171752),
 (171767, 171783),
 (172034, 172050),
 (172529, 172545),
 (174143, 174159),
 (177700, 177712),
 (181147, 181159),
 (181372, 181384),
 (181596, 181608),
 (181947, 181959),
 (182288, 182300),
 (182728, 182740),
 (182950, 182962),
 (183235, 183247),
 (183889, 183901),
 (184547, 184557),
 (185097, 185107),
 (185614, 185624),
 (187112, 18

In [46]:
# use the manning indexes to create a dictionary of dataframes for each manning region with LCMann Region Name as the key
mannings_dict={}
for x in mannings_index:
    mannings_dict[test_geo_df.iloc[x[0]][0].split("=")[-1]]=test_geo_df.iloc[x[0]:x[1]+1]

mannings_dict['WFCalcRi_1']

,0
154029,LCMann Region Name=WFCalcRi_1
154030,LCMann Region Table=17
154031,"NoData,0.04"
154032,"255,0.04"
154033,"Open Water,0.04"
154034,"Developed, Open Space,0.04"
154035,"Developed, Low Intensity,0.04"
154036,"Developed, Medium Intensity,0.04"
154037,"Developed, High Intensity,0.04"
154038,"Barren Land Rock-Sand-Clay,0.04"


In [26]:
# return the rows between the start and end of the mannings n values
mann_df0=test_geo_df.iloc[test_geo_df[test_geo_df[0].str.contains("LCMann Region Name")].index[0]:test_geo_df[test_geo_df[0].str.contains("LCMann Region Polygon")].index[0]]

In [25]:
# iterate through the main dataframe and get each mannings n dataframe like above
for x in plan_df.index:
    plan_df.loc[x,'mannings_n_df'] = pd.DataFrame(plan_df.loc[x,'geom_file'].read_text().split("\n")[plan_df.loc[x,'geom_file'].read_text().split("\n").index("LCMann Region Name"):plan_df.loc[x,'geom_file'].read_text().split("\n").index("LCMann Region Polygon")])
plan_df

NameError: name 'plan_df' is not defined

In [1]:
# class RASrun():

#     def compObserved_Sim(hdf_loc, observed_csv, gageVector, projection):
#         # Read in the observed data (this has to be prepared first)
#         observed = pd.read_csv(observed_csv, index_col=0, parse_dates=True)
#         # change to column names to strings
#         observed.columns = observed.columns.astype(str)

#         # Read in the gage shapefile and filter to the gage of interest
#         gageVec=gpd.read_file(gageVector)
#         # turn gageVec to a geodataframe
#         gageVec=gpd.GeoDataFrame(gageVec)

#         # Read in the boundary
#         boundary=RAS_Utility.get_2d_boundary(hdf_loc, projection)

#         # clip the gages to the boundary
#         gageVec=gageVec[gageVec.within(boundary.geometry[0])]

#         # reproject the gages to the same projection as the boundary
#         gageVec=gageVec.to_crs(boundary.crs, inplace=True)

#         # Read in the cell coordinates
#         cell_coords=RAS_Utility.get_cell_cords(hdf_loc)

#         # spatially join the gages to the cell coordinates
#         cell_gageVec=gpd.sjoin_nearest(gageVec, cell_coords, how='left')

#         # keep only the site id, cell id, and geometry
#         cell_gageVec=cell_gageVec[['SITE_ID','cell','geometry']]

#         # filter the rows to only the gage of interest
#         cell_gageVec=cell_gageVec[cell_gageVec.SITE_ID.isin(observed.columns)]

#         # create an empty dataframe to hold the simulated data
#         sim=pd.DataFrame(index=observed.index)
#         for cell in cell_gageVec.cell:
#             cell_gageVec.cell=np.nan

#         # Read in the simulated data
#         simulated=RAS_Utility.get_cell_wse(hdf_loc)

#         # fill the dataframe with the simulated data
#         for cell in cell_gageVec.cell:
#             sim[str(cell)]=simulated[cell].values

#         # assign the proper site id to the simulated data
#         sim.columns=cell_gageVec.set_index('cell').SITE_ID

#         # create a holomap of the observed and simulated data
#         calibrate_map=hv.HoloMap({site:hv.Curve(sim[site]).opts(color='red')*hv.Curve(observed[site]).opts(color='blue') for site in sim.columns}, kdims='Site')

#         return calibrate_map.opts(title='Observed vs. Simulated', xlabel='Date', ylabel='Water Surface Elevation (m)')
    
# viz = Viz